In [21]:
import os
import pandas as pd

import torch
import whisper
from transformers import pipeline

from transformers import logging
logging.set_verbosity_error()

In [22]:
# !pip install openai-whisper

In [23]:
device = "cuda" if torch.cuda.is_available() else "cpu"
torch_dtype = torch.bfloat16 if torch.cuda.is_available() else torch.float32


whisper_model = whisper.load_model("large")
whisper_model = whisper_model.to(device)

def whisper_trans(audio_path):
    """Transcribe with OpenAI Whisper (single file)"""
    with torch.no_grad():
        result = whisper_model.transcribe(audio_path)
    return result["text"]

In [24]:
task = "transcribe"

pipe_pathumma = pipeline(
    task="automatic-speech-recognition",
    model="nectec/Pathumma-whisper-th-large-v3",
    torch_dtype=torch_dtype,
    device=device,
)

pipe_pathumma.model.config.forced_decoder_ids = (
    pipe_pathumma.tokenizer.get_decoder_prompt_ids(
        language=None,
        task=task
    )
)

def pathumma_trans(audio_paths, batch_size=4):
    """Batch transcription with Pathumma Whisper"""
    outputs = pipe_pathumma(audio_paths, batch_size=batch_size)
    return [o["text"] for o in outputs]

In [25]:
folder_list = ["Lisa", "Bambam", "IU", "IVE"]
file_path_list = []

for folder in folder_list:
    sub_folder = folder + "/dataset"
    for file in os.listdir(sub_folder):
        folder_path = folder + "/dataset/" + file
        for wav_file in os.listdir(folder_path):
            if wav_file.endswith(".wav"):
                file_path = folder_path + "/" + wav_file
                file_path_list.append(file_path)

file_path_list[0]

'Lisa/dataset/5QmaVImK37w/SPEAKER_06_sent_0000.wav'

In [26]:
rows = []

# Pathumma ทำแบบ batch (เร็ว + GPU เต็ม)
pathumma_texts = pathumma_trans(file_path_list, batch_size=4)

for file_path, pathumma_text in zip(file_path_list, pathumma_texts):
    whisper_text = whisper_trans(file_path)

    speaker = file_path.split("/")[0]

    row_dict = {
        "file_path": file_path,
        "whisper_text": whisper_text,
        "pathumma_text": pathumma_text,
        "speaker": speaker
    }

    rows.append(row_dict)

    print(f"File: {file_path} - Whisper: {whisper_text} - Pathumma: {pathumma_text}")

File: Lisa/dataset/5QmaVImK37w/SPEAKER_06_sent_0000.wav - Whisper: คือเรื่องที่เราพูดกันวันเนี้ยมันมาจากใจแล้วหนูไม่เคยพูดที่ไหนมาก่อนNew Chapter ของ Lisa ใน 1 ปีที่ผ่านมา - Pathumma: คือเรื่องที่เราพูดกันวันนี้มันมาจากใจแล้วหนูไม่เคยพูดที่ไหนมาก่อน New Chapter ของลิซ่าใน 1 ปีที่ผ่านมา
File: Lisa/dataset/5QmaVImK37w/SPEAKER_06_sent_0001.wav - Whisper: มันก็คือละไฟหอดRoller Coaster Rideเอาจริงๆ ตอนนั้นยอมรับเลยว่า - Pathumma: มันก็คือระไฟเหาะเนอะรอเลอคอสเตอร์ไหว้เอาจริงจริงตอนนั้นยอมรับเลยว่า
File: Lisa/dataset/5QmaVImK37w/SPEAKER_06_sent_0002.wav - Whisper: เครียด - Pathumma: เครียด
File: Lisa/dataset/5QmaVImK37w/SPEAKER_06_sent_0003.wav - Whisper: เหมือนมันมองไม่เห็นอนาคต giving me any prior attentionกันอันไกลแค่แบบเดือนหน้า kin agianเราก็ไม่รู้ว่ามันจะออกมาเป็นรูป Anhain - Pathumma: เหมือนมันมองไม่เห็นอนาคตอันใกล้ แค่แบบเดือนหน้าเราก็ไม่รู้ว่ามันจะออกมาเป็นรูปแบบไหน เราไม่มีคนที่จะถามปรึกษา เรามีแค่ทีมเราแค่นั้น แต่มันไม่มีการันตีว่ามันจะโอเค มันจะเวิร์คใช่ไหมสรุปแล้ว
File: Lisa/data

In [27]:
df = pd.DataFrame(rows)
df.to_csv("transcription_results.csv", index=False)

In [28]:
df.head()

,file_path,whisper_text,pathumma_text,speaker
0,Lisa/dataset/5QmaVImK37w/SPEAKER_06_sent_0000.wav,คือเรื่องที่เราพูดกันวันเนี้ยมันมาจากใจแล้วหนู...,คือเรื่องที่เราพูดกันวันนี้มันมาจากใจแล้วหนูไม...,Lisa
1,Lisa/dataset/5QmaVImK37w/SPEAKER_06_sent_0001.wav,มันก็คือละไฟหอดRoller Coaster Rideเอาจริงๆ ตอน...,มันก็คือระไฟเหาะเนอะรอเลอคอสเตอร์ไหว้เอาจริงจร...,Lisa
2,Lisa/dataset/5QmaVImK37w/SPEAKER_06_sent_0002.wav,เครียด,เครียด,Lisa
3,Lisa/dataset/5QmaVImK37w/SPEAKER_06_sent_0003.wav,เหมือนมันมองไม่เห็นอนาคต giving me any prior a...,เหมือนมันมองไม่เห็นอนาคตอันใกล้ แค่แบบเดือนหน้...,Lisa
4,Lisa/dataset/5QmaVImK37w/SPEAKER_06_sent_0004.wav,เราทำเต็มที่แล้ว เราภูมิใจในของเราอืมแต่มันก็จ...,เราทำเต็มที่แล้วเราภูมิใจในของเราแต่มันก็จะมีบ...,Lisa
